# SVM

Neste lab iremos implementar uma versão do SVM com solução via pseudoinversa.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# magic para definer resolução como retina
%config InlineBackend.figure_format = 'retina'

## Exemplo da aula

Primeiro iremos reproduzir o exemplo da aula, com o seguinte dataset

|$x_1$|$x_2$|$y$|
| --- | --- | --- |
|3|3|$\circ$|
|2|3|$\circ$|
|2|1|$\square$|
|1|1|$\square$|

e seguindo a convenção $\circ = 1$ e $\square = -1$.

In [ ]:
# Vetores coluna
v1 = np.array([[3,3]]).T
v2 = np.array([[2,3]]).T
v3 = np.array([[2,1]]).T
v4 = np.array([[1,1]]).T
n = 4

rhs = np.array([[1,1,-1,-1]]).T

In [ ]:
# Para facilitar, iremos agrupar todos os vetores em uma matriz
M = np.hstack((...))

# adição do offset
M = np.vstack( (M, np.ones((1,n))) )
M

In [ ]:
# Agora, calculamos todos os produtos internos
P = ... @ ...
P

Calculados dos produtos internos $P$, podemos resolver o sistema para encontrar a normal $w$

$$P x = rhs$$

In [ ]:
# encontra a combinação que define w
x = np.round(... @ ..., 2)
x

In [ ]:
# calcular w
w = M @ x
w

In [ ]:
# função de predição
predicao = lambda x : w.T @ x

In [ ]:
# testando a predição
predicao( M ).flatten(), rhs.flatten()

## Heart Disease

Base de dados sobre doenças cardíacas da UCI Irvine Machine Learning Repository. No [site](https://archive.ics.uci.edu/dataset/45/heart+disease) é possível ver a descrição da base, comentários e o desempenho dos modelos.

O objetivo é prever a última coluna "present" que indica se possui a doença.

In [ ]:
db = pd.read_csv("heart.csv")
db.present = db.present.replace(0,-1)

# 80% dos dados serão utilizados para treino
n_training = int(db.shape[0] * 0.8)

# o ideal é amostrar os dados e não pegar os primeiros 80% como abaixo
training = db.loc[:n_training]
test = db.loc[n_training:]

In [ ]:
# colunas com as features e a classe
db.columns.values

In [ ]:
# plota chol contra age, com cores em present para verificar a separabilidade
db.plot.scatter(x="chol", y="age", c="present", colormap="winter", alpha=0.5)

In [ ]:
def ret_desvios(dataset):
  # dataframe com apenas as features 1:-1
  F = dataset.iloc[:, 1:-1]
  rhs = dataset.present

  # É importante normalizar os dados antes de aplicar SVM, para deixarmos todos os dados na mesma escala
  D = F - F.mean()
  M = D / D.std()
  return M

M = ret_desvios( training )
M.head(3)

In [ ]:
# Calculamos todos os produtos internos
P = ... @ ...
# encontra a combinação que define w
x = np.linalg.pinv(P) @ training.present
# calcula w
w = M.T @ x
w

In [ ]:
# função de predição
predicao = lambda x : (w.T @ x) >= 0

# funções de avaliação
acuracia = lambda predicao, rhs : np.sum(predicao == rhs) / len(rhs)
precisao = lambda predicao, rhs : np.sum(predicao[rhs == 1] == rhs[rhs == 1]) / np.sum(predicao == 1)

Agora iremos realizar a predição para o conjunto de treino e depois para o conjunto de teste

In [ ]:
# aplicar predicao
pred = predicao(M.T).astype(int) * 2 - 1

# erros
acuracia(pred, training.present), precisao(pred, training.present)

In [ ]:
# verificar dados de teste
M_t = ret_desvios( test )
pred = predicao(M_t.T).astype(int) * 2 - 1

# erros
acuracia(pred, test.present), precisao(pred, test.present)

## Utilizando SVC

Podemos utilizar a implementação do support vector classifier do sklearn, que facilita todo este processo

In [ ]:
from sklearn.svm import SVC

In [ ]:
# modelo com kernel linear
model = SVC(kernel="linear")
model.fit(M, training.present)

# predição
pred = model.predict(M)

# erros
acuracia(pred, training.present), precisao(pred, training.present)

In [ ]:
# validação
pred = model.predict(M_t)

# erros
acuracia(pred, test.present), precisao(pred, test.present)

## Com Kernel polinomial

In [ ]:
# modelo com kernel linear
model = SVC(kernel="poly", degree=2)
model.fit(M, training.present)

# predição
pred = model.predict(M)

# erros
acuracia(pred, training.present), precisao(pred, training.present)

In [ ]:
# validação
pred = model.predict(M_t)

# erros
acuracia(pred, test.present), precisao(pred, test.present)

## Com Kernel RBF

In [ ]:
# modelo com kernel linear
model = SVC(kernel="rbf")
model.fit(M, training.present)

# predição
pred = model.predict(M)

# erros
acuracia(pred, training.present), precisao(pred, training.present)

In [ ]:
# validação
pred = model.predict(M_t)

# erros
acuracia(pred, test.present), precisao(pred, test.present)

## Com Kernel RBF e ajuste de soft margin

In [ ]:
# modelo com kernel linear
model = SVC(kernel="rbf", C=10) # verifique a doc para ajustar C
model.fit(M, training.present)

# predição
pred = model.predict(M)

# erros
acuracia(pred, training.present), precisao(pred, training.present)

In [ ]:
# validação
pred = model.predict(M_t)

# erros
acuracia(pred, test.present), precisao(pred, test.present)